This notebook aims to experiment loading of saved models. The saved models used here is from notebook 12_custom_models_and_training.ipynb, section - Saving/Loading Models with Custom Objects.


### Importing libraries

In [3]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## to make the notebook output stable
np.random.seed(42)
tf.random.set_seed(42)

### Dataset for our experiment

In [2]:
X, y = fetch_california_housing(return_X_y=True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=42)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_valid_scaled = scaler.transform(X_valid)

## Custom LOSS

The three case dealt here are:
 - model that was compiled with a custom loss function (testing .keras and .h5 format)
 - model that was compile with a paremeterized custom loss function (testing .h5 format)
 - model that was compiled with custom loss subclasssed (testing .keras, .h5, savedModel format)

#### Loading a saved model that used a custom loss function

Test saved model `(.keras)` that used custom loss function

In [18]:
model_custom_loss_function_dot_keras = tf.keras.models.load_model('/content/my_model_custom_loss_huber.keras')
model_custom_loss_function_dot_keras.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 1s 3ms/step - loss: 0.1980 - mae: 0.4882 - mse: 0.4682


[0.19795940816402435, 0.48822590708732605, 0.4682220220565796]

Test saved model`(.h5)` that used custom loss function

In [19]:
model_custom_loss_function = tf.keras.models.load_model('/content/my_model_custom_loss_huber.h5')

In [20]:
model_custom_loss_function.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 1s 3ms/step - loss: 0.1980 - mae: 0.4882 - mse: 0.4682


[0.19795940816402435, 0.48822590708732605, 0.4682220220565796]

Test saved model (SavedModel format) that used custom loss function

In [33]:
#load model and try evaluation
model_saveModelformat = keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/HML/my_model_custom_loss_huber')
model_saveModelformat.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 1s 2ms/step - loss: 0.1980 - mae: 0.4882 - mse: 0.4682


[0.19795940816402435, 0.48822590708732605, 0.4682220220565796]

Loosk like we don't need to specify custom_object when loading model using a custom loss function

Test saved model (.h5) that used parameterized custom loss function

In [23]:
model_custom_loss_function_parameterized = tf.keras.models.load_model('/content/my_model_custom_loss_parameterized.h5')
model_custom_loss_function_parameterized.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.1916 - mae: 0.4829


[0.19164715707302094, 0.4828616976737976]

Lets try loading the model with custom function object specified. For this you need the custom function available in the session

In [9]:
# define huber loss

def huber_loss(y_true, y_pred):
  error = y_true - y_pred
  is_small_error = tf.abs(error) < 1
  squared_loss = tf.square(error) / 2
  linear_loss = tf.abs(error) - 0.5

  return tf.where(is_small_error, squared_loss, linear_loss)

In [21]:
model_custom_loss_function_dot_keras_custom_object = tf.keras.models.load_model('/content/my_model_custom_loss_huber.keras',
                                                                                custom_objects={'huber_loss': huber_loss})

In [22]:
model_custom_loss_function_dot_keras_custom_object.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 0s 2ms/step - loss: 0.1980 - mae: 0.4882 - mse: 0.4682


[0.19795940816402435, 0.48822590708732605, 0.4682220220565796]

So that also works, but seems that we need to track of an additional step of having the custom function available for the model to load with custom objects

#### Loading a saved model that used a subclassed loss function

Test saved model `(.keras)` that used custom loss subclassed

In [12]:
model_custom_loss_subclassed_dot_keras = tf.keras.models.load_model('/content/my_model_custom_loss_subclassed.keras')

ValueError: ignored

It is must to load model with custom objects specified, the reason being `.keras/HDF5` formats uses object `configs` to save the model architecture

In [13]:
model_customloss_subclassed_dotkeras = tf.keras.models.load_model('my_model_custom_loss_subclassed.keras',
                                                               custom_objects={'HuberLoss':HuberLoss})

NameError: ignored

subclassed code should be available

In [15]:
from tensorflow import keras
class HuberLoss(keras.losses.Loss):

    def __init__(self, threshold=1.0, **kwargs):
        self.threshold=threshold
        super().__init__(**kwargs)
    ##call(): Contains the logic for loss calculation using y_true, y_pred
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = error < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold':self.threshold}

In [30]:
model_customloss_subclassed_dotkeras = tf.keras.models.load_model('my_model_custom_loss_subclassed.keras',
                                                               custom_objects={'HuberLoss':HuberLoss})

In [31]:
model_customloss_subclassed_dotkeras.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 1s 2ms/step - loss: 0.2398 - mae: 0.5181


[0.23983554542064667, 0.5180786848068237]

In [24]:
model_customloss_subclassed_dotkeras.get_config() # model configuration

{'name': 'sequential_1',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 8),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_2_input'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_2',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 8),
    'units': 32,
    'activation': 'selu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'LecunNormal',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_3',
    'trainable': True,
    'dtype': 'float32',
    'units': 1,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_ini

model compile configuration. Note the **loss object, its config**

In [26]:
model_customloss_subclassed_dotkeras.get_compile_config()

{'optimizer': {'module': 'keras.optimizers.experimental',
  'class_name': 'Nadam',
  'config': {'name': 'Nadam',
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'jit_compile': False,
   'is_legacy_optimizer': False,
   'learning_rate': 0.0010000000474974513,
   'beta_1': 0.9,
   'beta_2': 0.999,
   'epsilon': 1e-07},
  'registered_name': None},
 'loss': {'module': '__main__',
  'class_name': 'HuberLoss',
  'config': {'reduction': 'auto', 'name': None, 'threshold': 2.0},
  'registered_name': 'HuberLoss'},
 'metrics': [[{'module': 'keras.metrics',
    'class_name': 'MeanMetricWrapper',
    'config': {'name': 'mae',
     'dtype': 'float32',
     'fn': {'module': 'keras.losses',
      'class_name': 'function',
      'config': 'mean_absolute_error',
      'registered_name': 'mean_absolute_error'}},
    'registered_name': None}]],
 'loss_weights': None,
 'weigh

In [43]:
# get the threshold attribute
model_customloss_subclassed_dotkeras.loss.threshold

2.0

.h5

In [28]:
model_customloss_subclassed_h5 = tf.keras.models.load_model('/content/my_model_custom_loss_subclassed.h5',
                                                               custom_objects={'HuberLoss':HuberLoss})

In [29]:
model_customloss_subclassed_h5.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 1s 3ms/step - loss: 0.2398 - mae: 0.5181


[0.23983554542064667, 0.5180786848068237]

In [42]:
model_customloss_subclassed_h5.loss.threshold

2.0

SavedModel

In [34]:
model_customloss_subclassed_savedModel = keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/HML/my_model_custom_loss_subclassed',
                                                                 custom_objects={'HuberLoss':HuberLoss})

In [35]:
model_customloss_subclassed_savedModel.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.2398 - mae: 0.5181


[0.23983554542064667, 0.5180786848068237]

In [38]:
model_customloss_subclassed_savedModel.fit(X_train_scaled, y_train,
                                           epochs=2,
                                           validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 3s 2ms/step - loss: 0.2348 - mae: 0.5069 - val_loss: 0.2404 - val_mae: 0.4929
Epoch 2/2
363/363 [==============================] - 1s 3ms/step - loss: 0.2232 - mae: 0.4934 - val_loss: 0.2614 - val_mae: 0.4852


In [41]:
model_customloss_subclassed_savedModel.loss.threshold

2.0

## Other Custom Functions (e.g., used in model layers)